<a href="https://colab.research.google.com/github/sophiapyx/IAT360-Computer-Vision-Project---Group-36/blob/main/Computer_Vision_Project_Group_36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import json
from PIL import Image

base_dir = "Desktop/datasets_cat"
output_dir = os.path.join(base_dir, "converted_yolo")

class_map = {"pain": 1, "no_pain": 0}
valid_exts = (".png", ".jpg", ".jpeg")

os.makedirs(output_dir, exist_ok=True)
for cls in class_map:
    os.makedirs(os.path.join(output_dir, cls, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, cls, "labels"), exist_ok=True)

def convert_bbox_to_yolo(bbox, img_w, img_h):
    x_min, y_min, x_max, y_max = bbox
    x_c = (x_min + x_max) / 2.0 / img_w
    y_c = (y_min + y_max) / 2.0 / img_h
    w   = (x_max - x_min) / img_w
    h   = (y_max - y_min) / img_h
    return x_c, y_c, w, h

for cls_name, cls_id in class_map.items():
    img_dir = os.path.join(base_dir, cls_name, "images")
    label_dir = os.path.join(base_dir, cls_name, "labels")
    out_img_dir = os.path.join(output_dir, cls_name, "images")
    out_lbl_dir = os.path.join(output_dir, cls_name, "labels")

    for fname in os.listdir(img_dir):
        lf = fname.lower()
        if not lf.endswith(valid_exts):
            continue

        base = os.path.splitext(fname)[0]
        img_path = os.path.join(img_dir, fname)
        json_path = os.path.join(label_dir, base + ".json")


        with Image.open(img_path) as im:
            w, h = im.size


        with open(json_path, "r") as f:
            data = json.load(f)
        bbox = data["bounding_boxes"]


        x_c, y_c, bw, bh = convert_bbox_to_yolo(bbox, w, h)
        with open(os.path.join(out_lbl_dir, base + ".txt"), "w") as f:
            f.write(f"{cls_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}\n")


        with Image.open(img_path) as im:
            im.save(os.path.join(out_img_dir, fname))


In [14]:
!pip3 install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.220 🚀 Python-3.13.7 torch-2.9.0 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 124.6/228.3 GB disk)


In [15]:
# Run inference on an image with YOLOv8n

# Detection:
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg'
!yolo predict model=yolov8n.pt source='https://www.thoughtco.com/thmb/Hw83C2QW8dWeDUt7R4pYyPke3HI=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/GettyImages-547031277-58ef97803df78cd3fc724e24.jpg' save=True
!yolo predict model=yolov8n.pt source=0

# Segmentation:
!yolo segment predict  model=yolov8s-seg.pt source='https://ultralytics.com/images/zidane.jpg'

#pose detection:
!yolo classify predict  model=yolov8s-cls.pt source='https://ultralytics.com/images/zidane.jpg'


#add arguments of "save and show"

WARNING ⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt... <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)>
######################################################################### 100.0%
Ultralytics 8.3.220 🚀 Python-3.13.7 torch-2.9.0 CPU (Apple M2)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

WARNING ⚠️ Download failure, retrying 1/3 https://ultralytics.com/images/zidane.jpg... <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)>
######################################################################### 100.0%
image 1/1 /Users/sophia/zidane.jpg: 384x640 2 persons, 1 tie, 57.4ms
Speed: 5.9ms preprocess, 57.4ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /Users/sophia/runs/detect/predict
💡 Learn mor